In [ ]:
# running and benchmarking scnaorama

In [1]:
import scanorama

import numpy as np
from scipy.io import mmread


In [2]:
#hsc_ss2 = np.load("/Users/jzahiri/Documents/Sales/SingleCell/Scanorama/Data/data/hsc/hsc_ss2.npz")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jzahiri/Documents/Sales/SingleCell/Scanorama/Data/data/hsc/hsc_ss2.npz'

In [16]:
for mtx in hsc_ss2:
    print(mtx)
print(hsc_ss2.files)
print(hsc_ss2['X'].shape)
print(hsc_ss2['genes'].shape)
hsc_ss2['X'][1:10,:]
hsc_ss2['genes']

# genes is a vector of gene names
# X is the matrix of expressions (?) with dimensions sample*genes

X
genes
['X', 'genes']
(774, 39109)
(39109,)


array(['GNAI3', 'PBSN', 'CDC45', ..., 'GM43124', 'GM43323', 'GM7792'],
      dtype='<U15')

## mtx and npz files
It seems that mtx are related to sparce matrix in scipy and npz files are related to zipped archives of the numpy ndarrays.

# running scanorama

Running examples from https://github.com/brianhie/scanorama#full-tutorial
There are lots of examples in bin folder. All examples assume that the data are in bin/data/ folder.
I ran these example mouse_brain.py. It needs gene name (genes.tsv), matrix.mtx along with the cell labels (which is mouse_brain_cluster.txt). It generates two files: tab.npz and tab.genes.txt
However, data can be in different format, e.g in hdf5 see process.py

 Data sets must be numpy array or scipy.sparse.csr_matrix

Note:
You can merge the datasets based on union of genes or intersect of genes. By default intersect of genes is used. In this case we have to have some common genes across all datasets otherwise: "Error: No genes found in all datasets, exiting..."

In [2]:
import os

import numpy as np
from scanorama import *
from scipy.sparse import vstack
from sklearn.preprocessing import normalize, LabelEncoder
import sys
from time import time
from bin.process import load_names, process

In [4]:
data_names = [
    'bin/data/mouse_brain/dropviz/Cerebellum_ALT',
    'bin/data/mouse_brain/dropviz/Cortex_noRep5_FRONTALonly',
    'bin/data/mouse_brain/dropviz/Cortex_noRep5_POSTERIORonly',
    'bin/data/mouse_brain/dropviz/EntoPeduncular'
]

# process(data_names, min_trans=100)
# it saves files in npz format if they are "mtx", "h5", or "txt"|"tsv"|"txt.gz"|"tsv.gz"
# if data_names are folders then it assumes that each folder contains a mtx file named "matrix.mtx", and a tsv file for gene names "genes.tsv"; if it is a file name then it checks different file types ("h5", or "txt"|"tsv"|"txt.gz"|"tsv.gz")


# "load_names" (which calls "load_data" which uses scipy.sparse.load_npz for loading data) assumes that datasets are npz files (any full path of a ".npz" file or a folder containing "tab.npz" and "tab.genes.txt" files; these files are created by "process" function from other data types). It loads datasets and normalize them based on "norm" flag. If you are going to do the run "integrate" it does the normalization by calling the "process_data" so you can set "norm" to False.
# signature: load_names(data_names, norm=True, log1p=False, verbose=True).
t0 = time()
datasets, genes_list, n_cells = load_names(data_names)
print('Load time: {:.3f}s'.format(time() - t0))


Loaded bin/data/mouse_brain/dropviz/Cerebellum_ALT with 24247 genes and 76250 cells
Loaded bin/data/mouse_brain/dropviz/Cortex_noRep5_FRONTALonly with 29463 genes and 194019 cells
Loaded bin/data/mouse_brain/dropviz/Cortex_noRep5_POSTERIORonly with 27720 genes and 116432 cells
Loaded bin/data/mouse_brain/dropviz/EntoPeduncular with 22631 genes and 33602 cells
Found 420303 cells among all datasets
Load time: 7.710s


In [ ]:
t0 = time()
datasets_dimred, datasets, genes = correct(
    datasets, genes_list, ds_names=data_names,
    return_dimred=True, batch_size=BATCH_SIZE,
)
print('Batch corrected panoramas: {:.3f}s'.format(time() - t0))


After bin/data/mouse_brain/dropviz/Cerebellum_ALT: 24246 genes
After bin/data/mouse_brain/dropviz/Cortex_noRep5_FRONTALonly: 23758 genes
After bin/data/mouse_brain/dropviz/Cortex_noRep5_POSTERIORonly: 23204 genes
After bin/data/mouse_brain/dropviz/EntoPeduncular: 21080 genes
Found 21080 genes among all datasets
[[0.         0.14142951 0.05346885 0.14005119]
 [0.         0.         0.6532053  0.55529433]
 [0.         0.         0.         0.44235462]
 [0.         0.         0.         0.        ]]
Processing datasets bin/data/mouse_brain/dropviz/Cortex_noRep5_FRONTALonly <=> bin/data/mouse_brain/dropviz/Cortex_noRep5_POSTERIORonly


In [ ]:
t0 = time()
integrated, corrected, genes = scanorama.correct(datasets, genes_list, return_dimred=True)
print('Batch corrected and integration in {:.3f}s'
      .format(time() - t0))